# RDM - Tools for supporting (large) data transfers - excerpts 

This is an experiment -- a notebook to capture practices that support the acceleration of large data transfer (10s of TBs; millions of files) and perhaps automate some or all of them.

Run this notebook locally on the machine that has to data to transfer. It's written for Mac OS X (Unix/bash). It can probably be easily modified for Linux boxes.

#### Last revised: 27 March 2018

By: Rick

To set automagic on, follow the command with the argument '1'

In [ ]:
%automagic 1

## Sample data

To demonstrate the data profiling tools, we'll need a directory with a ton of files. I have one with Library of Congress data on my computer. Locate an appropriate set of sample data on yours.

In [ ]:
cd /Users/rjaffe/Jupyter_notebooks/RDM_datatransfer/test_data/LOC_Data/batch_az_acacia_ver01/

In [ ]:
ls

...count the files (`-type f`), starting from the present working directory.

In [ ]:
!find . -type f | wc -l

### Looping one-liners

We can use looping (and other iterating techniques) to profile the files in a directory tree, recursively from the present working directory.

Here's a one-liner adapted from a command that John White used recently on Savio. It uses the `ls` command to list the subdirectories in the current directory, then lists the items in each subdirectory and passes the results to `wc` to count them.

In [ ]:
!for i in `ls`; do echo $i; find $i | wc -l; done

Some researchers have found that certain file types -- .mat files, for one, apparently -- take a long time to transfer. This variation counts the number of files in each directory with a particular filename extension. (I use .xml as an example only because my test data set contains a number of .xml files.)

In [ ]:
!for i in `ls`; do echo $i; find $i  -type f -name *.xml | wc -l; done

Find all files with a given filename extension (i.e, of a specific file type).

In [ ]:
!find ./sn89053851 -type f -name *.xml  #note the use of '*' as a wildcard, meaning 'any set of characters'

This one-liner loops through a given set of values, calling the `find` command to identify files whose size is greater than each value, then pipes that information to `wc` to count those files.  Here, M = megabytes and G = gigabytes.

The output shows the number of files of each size.

In [ ]:
!for size in +0 +1M +10M +1G +10G ; do echo "$size"; find . -type f -size $size | wc -l; done

### Tree

The `tree` command diagrams the structure of your local file tree. The first argument points to the top of the tree you want to draw. The `-d` flag means 'show only directories' (i.e., not files); the `-o` option writes the output to a local file, which we'll name 'LOCTree.txt'. Writing to a file is valuable because the output can get quite long. 

I didn't have the 'tree' package installed on my Mac. I needed to use a package manager to download and install. I use Homebrew (https://brew.sh/), but there are others. To install 'tree' using Homebrew, the command is `brew install tree`. More documentation can be found at https://docs.brew.sh/Manpage.

In [ ]:
!tree ./ -d -o LOCTree.txt

Let's read the file into the notebook and take a look at the file tree.

In [ ]:
cat "LOCTree.txt"

##  Running shell scripts

Can we run a shell script? Yes. (Of course.)

For instructions, see: https://www.quora.com/How-do-I-execute-bash-scripts-via-IPython-Jupyter-notebook.

Let's demonstrate how-to by running a useful script...one that finds all directories with greater than 2000 files and displays the size and name of those directories. 
 
 First, let's locate the script: it's called 'dirs-gt-2000-du.sh' and it's in the same directory as this notebook. See In[2], above.

In [ ]:
cd /Users/rjaffe/Jupyter_notebooks/rdm_datatransfer

In [ ]:
ls

Running scripts within the notebook requires starting a new shell process. There's a library for that: `subprocess`.

In [ ]:
import subprocess

The script takes an argument: the directory that we will profile. We pass both the script name and directory to be profiled as arguments to the subprocess function. 

Be patient: this will take a while!

In [ ]:
dirinfo = subprocess.run(['./dirs-gt-2000-du.sh', '/Users/rjaffe/Jupyter_notebooks/rdm_datatransfer/test_data/LOC_Data/batch_az_acacia_ver01', '/dev/null'], stdout=subprocess.PIPE)
print(dirinfo.stdout.decode("utf-8", "strict"))

voilà!

## Challenge

PROBLEM: RDM has consulted on several cases in which researchers were trying to upload millions of files to Box or Drive and encountering v-e-r-y slow transfer rates. Also, we've encountered at least two common research tools (Opera Phenix microscopes and FMRIB FSL analysis software) that can produce individual directories containing greater than 20,000 files -- the limit that Box can display without throwing a "File can't be found' error in response to the MLSD command used by FileZilla and other FTP clients.

CONSTRAINTS:

• Copying zillions of tiny files ("ZOTS") takes much longer than many fewer, larger files. 

• Google Drive and Box both throttle bulk uploads to as few as two files per second. 

• Box has a per-file file size limit of 15GB.  

CHALLENGE 1: Can we use these tools to devise a strategy for identifying the ends of the tree -- where the number of files tend to be great -- and then tarring or zipping up just those directories? The goal is to enable faster transfer rates to Box and Drive, while allowing researchers to download to their analysis environments only those portions of the data that they need for any given analysis.

CHALLENGE 2: Can we run this notebook from the cloud and point it at a local machine with internal or mounted storage to transfer?

# Using rclone

Now let's try running rclone to copy a file from the local machine to Google Drive.

First, navigate to the file...

In [ ]:
cd /Users/rjaffe/Documents/RDM/TestData

In [ ]:
ls

In [ ]:
cd Transfer-test-no-subfolders/

In [ ]:
ls -al

Now run rclone to copy one of the files in this directory to a pre-configured drive account.

In [ ]:
!rclone copy /Users/rjaffe/Documents/RDM/TestData/Transfer-test-no-subfolders/Picture\ 8.png rclone-rdmconsult-bdrive:test_from_notebook

Check the drive account in a separate browser to confirm that the file actually transferred. (It did!)

# Using Globus and the Globus API

...to be added.